In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import datapackage
#import tabula
#from tabula import read_pdf

# Data Cleaning 

## Country data 
Additional data required to join WDI & IFS data on common country code

In [ ]:

WOE_Dataset = pd.read_csv('IMF_country_codes.csv')
def WEO_country(WOE_Dataset):
    # Get ride of unused data
    WOE_Dataset.drop(columns= WOE_Dataset.columns[4:] , inplace=True)
    #rename columns
    old_columns = WOE_Dataset.columns
    New_columns = WOE_Dataset.iloc[0].to_list()
    rename_map = dict(zip(old_columns, New_columns))
    WOE_Dataset.rename(columns = rename_map, inplace = True)

    #Drop column 0 - the true header
    WOE_Dataset = WOE_Dataset.drop(0).copy()
    return WOE_Dataset 

In [ ]:
cntry_map = pd.read_csv('IMF_country_codes.csv')
cntry_map = CountrieWEO_country(cntry_map)

,IMF Code,ISO Code,Country,Currency
1,512,AFG,Afghanistan,Afghan Afghani
2,914,ALB,Albania,Albanian lek
3,612,DZA,Algeria,Algerian dinar
4,614,AGO,Angola,Angolan kwanza
5,311,ATG,Antigua and Barbuda,Eastern Caribbean dollar
...,...,...,...,...
185,299,VEN,Venezuela,Venezuelan bol�var fuerte
186,582,VNM,Vietnam,Vietnamese dong
187,474,YEM,Yemen,Yemeni rial
188,754,ZMB,Zambia,Zambian kwacha


### World Develpment Indicators (WDI) Data - The World Bank
Notes on outstanding work: 1 - Make Data DateTime Object. 2 - concat the Indicator code to the title name

In [ ]:
wdi_sample = pd.read_csv('WDI - sample.csv')

In [ ]:

#Clean titles
#Create Dictionary of column header mapping & use rename function
old_columns = wdi_sample.columns[4::]
yr_columns = wdi_sample.columns[4::].str[:4]
yr_columns_dict = dict(zip(old_columns, yr_columns))
wdi_sample.rename(columns = yr_columns_dict, inplace = True)

#Country Selection
cntrys = ['South Africa', 'Afghanistan']
wdi_sample[wdi_sample['Country Name'].isin(cntrys)]

#Reform Pivotted Data
wdi_sample.drop(['Series Code'], axis=1, inplace = True)
wdi_sample = wdi_sample.melt(id_vars=['Country Name','Country Code','Series Name'], var_name='Year', value_name='Value' ).copy()
# Note, this won't work unless you've got selected countries! 
wdi_sample = wdi_sample.pivot(index=['Year', 'Country Name' ,'Country Code'], columns='Series Name', values='Value').reset_index().copy()

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
wdi_sample

#### World Development Indicators data Cleaning Function



In [ ]:
def clean_wdi_data(dataset, countries, date_from = 1980, date_to = 2002 ):
    '''
    dataset {df}: WDI Data sample
    countries {List}: List of countries to be inclided in Dataset
    date_from {int}: Lowest year - default 1980
    date_to {int}: Highest Yeat - default 2020

    Output{df}: Year, Country Name , Country Code, [Series Names]

    '''
    #Clean titles
    #Create Dictionary of column header mapping & use rename function
    old_columns = dataset.columns[4::]
    yr_columns = dataset.columns[4::].str[:4]
    yr_columns_dict = dict(zip(old_columns, yr_columns))
    dataset.rename(columns = yr_columns_dict, inplace = True)
    
    #Country Selection
    dataset = dataset[dataset['Country Name'].isin(countries)].copy()
    
    #Reform Pivotted Data
    dataset.drop([ 'Series Code'], axis=1, inplace = True)
    dataset = dataset.melt(id_vars=['Country Name','Country Code','Series Name'], var_name='Year', value_name='Value' ).copy()
    dataset = dataset.pivot(index=['Year', 'Country Name', 'Country Code'], columns='Series Name', values='Value').reset_index().copy()

    return dataset

In [ ]:
test = pd.read_csv('WDI - sample.csv')
hun = ['Hungary', 'South Africa']
clean_wdi_data(test, hun, date_from = 1980, date_to = 2002 )

Series Name,Year,Country Name,Country Code,Broad money growth (annual %),Broad money to total reserves ratio,Gross capital formation (constant 2010 US$),Real effective exchange rate index (2010 = 100)
0,1980,Hungary,HUN,..,..,..,..
1,1980,South Africa,ZAF,22.8507513885544,5.47953147000172,41470623355.0728,165.188043881133
2,1981,Hungary,HUN,..,..,..,..
...,...,...,...,...,...,...,...
79,2019,South Africa,ZAF,6.11024523226612,4.73151814615898,81243117953.9007,78.1427521961595
80,2020,Hungary,HUN,20.8940730816157,..,..,85.4569190386896
81,2020,South Africa,ZAF,9.42732646118054,..,..,70.9184570461496


### International Financial Statistics (IFS DB) data - International Monetary Fund
Notes on outstanding work: 1 - Make Data DateTime Object. 2 - concat the Indicator code to the title name. 3 - Need to Edit the Indicators being used. 

| Field      | Indicator Code | Source    |
| :---        |    :----:   |    :---: |
| Header      | Title       | Here's this   |
| Paragraph   | Text        | And more      |

In [ ]:
IFS_sample = pd.read_csv('IFS sample.csv')


In [ ]:
IFS_DB =  pd.read_csv('IFS_timeSeries.csv')

,Country Name,Country Code,Indicator Name,Indicator Code,Attribute,1920,1920Q1,1920Q2,1920Q3,1920Q4,...,2020M12,2021,2021Q1,2021M1,2021M2,2021M3,2021Q2,2021M4,2021M5,Base Year
0,"Afghanistan, Islamic Rep. of",512,"Fund Accounts, UFC & Loans, SDRs (Test)",HPU_XDR,Value,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test
1,"Afghanistan, Islamic Rep. of",512,"Fund Accounts, UFC & Loans, SDRs (Test)",HPU_XDR,Value,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.812300e+08,NaN,NaN,NaN,NaN,NaN,NaN,Test
2,"Afghanistan, Islamic Rep. of",512,"Fund Accounts, UFC & Loans, SDRs",HPU_XDR,Value,NaN,NaN,NaN,NaN,NaN,...,2.812300e+08,NaN,NaN,2.812300e+08,2.812300e+08,2.812300e+08,NaN,2.788300e+08,NaN,NaN
3,Advanced Economies,110,"Fund Accounts, Outstanding Loans, SDRs",HPL_XDR,Value,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Advanced Economies,110,"Fund Accounts, Outstanding Loans, SDRs",HPL_XDR,Value,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Advanced Economies,110,"Fund Accounts, Outstanding Loans, SDRs",HPL_XDR,Value,NaN,NaN,NaN,NaN,NaN,...,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN
6,Angola,614,"Fund Accounts, SDR Holdings as % of Allocation...",HPHSPA_PA,Value,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Angola,614,"Fund Accounts, SDR Holdings as % of Allocation...",HPHSPA_PA,Value,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6.535922e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Angola,614,"Fund Accounts, SDR Holdings as % of Allocation...",HPHSPA_PA,Value,NaN,NaN,NaN,NaN,NaN,...,6.854873e+01,NaN,NaN,6.813826e+01,6.535922e+01,6.535922e+01,NaN,6.535754e+01,NaN,NaN


In [ ]:
# clean & working Code 

# Select Countries Required
cntrys = ['Afghanistan, Islamic Rep. of', 'Angola']
IFS_sample = IFS_sample[IFS_sample['Country Name'].isin(cntrys)].copy()

#Concat Base year 
IFS_sample['Indicator Name'] = np.where(IFS_sample['Base Year'].isnull(),IFS_sample['Indicator Name'], IFS_sample['Indicator Name'] + " (" + IFS_sample['Base Year'] +")").copy()
# Filter by then concat the Indicator Code
IFS_sample['Indicator Name'] = IFS_sample['Indicator Name'] + " - " + IFS_sample['Indicator Code']
required_indicators = ['HPHSPA_PA']
IFS_sample = IFS_sample[IFS_sample['Indicator Code'].isin(required_indicators)].copy()
#Drop Base Year & Indicator Code
IFS_sample.drop(['Base Year', 'Indicator Code'], axis=1, inplace = True)

#Melt Data 
IFS_sample.drop([ 'Attribute'], axis=1, inplace = True)
IFS_sample = IFS_sample.melt(id_vars=['Country Name','Country Code','Indicator Name'], var_name='Year', value_name='Value' ).copy()


#Cut data Range
#Frequency - Annual - Done
'''
non_ann = ['M', 'Q']
IFS_sample = IFS_sample[~IFS_sample.Year.str.contains('|'.join(non_ann))].copy()
'''
#Frequency - Quarterly 
IFS_sample = IFS_sample[IFS_sample.Year.str.contains('Q')].copy()
IFS_sample.drop_duplicates(inplace=True)

#Pivot Indicators to Columns
IFS_sample = IFS_sample.groupby(['Year', 'Country Name', 'Country Code', 'Indicator Name']).agg({'Value':'sum'}).reset_index().copy()
IFS_sample = IFS_sample.pivot(index=['Year', 'Country Name', 'Country Code'], columns='Indicator Name', values='Value').reset_index().copy()

In [ ]:
IFS_sample.head(30)

Indicator Name,Year,Country Name,Country Code,"Fund Accounts, SDR Holdings as % of Allocation, Percent per annum - HPHSPA_PA"
0,1920Q1,Angola,614,0.0
1,1920Q2,Angola,614,0.0
2,1920Q3,Angola,614,0.0
...,...,...,...,...
27,1926Q4,Angola,614,0.0
28,1927Q1,Angola,614,0.0
29,1927Q2,Angola,614,0.0


In [ ]:
def clean_IFS_data(dataset, countries, frequency, date_from, Series_Codes , date_to):
    '''
    dataset {df}: WDI Data sample
    countries {List}: List of countries to be inclided in Dataset
    date_from {int}: Lowest year - default 1980
    date_to {int}: Highest Yeat - default 2020

    Output{df}: 'Year', 'Country Name', 'Country Code', [Series Names]

    '''
    #Country Selection
    dataset = dataset[dataset['Country Name'].isin(countries)].copy()   

    #Concat Base year & Drop Base Year Column
    dataset['Indicator Name'] = np.where(dataset['Base Year'].isnull(),dataset['Indicator Name'], dataset['Indicator Name'] + " (" + dataset['Base Year'] +")").copy()
    
    # Filter by then concat the Indicator Code
    dataset = dataset[dataset['Indicator Code'].isin(Series_Codes)].copy()
    dataset['Indicator Name'] = dataset['Indicator Name'] + " - " + dataset['Indicator Code']
    
    dataset.drop(['Base Year', 'Indicator Code'], axis=1, inplace = True)

    #Melt Data 
    dataset.drop([ 'Attribute'], axis=1, inplace = True)
    dataset = dataset.melt(id_vars=['Country Name','Country Code','Indicator Name'], var_name='Year', value_name='Value' ).copy()

    #Frequency - Annual - Done
    if frequency == 'Y':
        non_ann = ['M', 'Q']
        dataset = dataset[~dataset.Year.str.contains('|'.join(non_ann))].copy()
    elif frequency == 'Q':
        dataset = dataset[dataset.Year.str.contains('Q')].copy()
    elif frequency == 'M':
        dataset = dataset[dataset.Year.str.contains('M')].copy()

    #Pivot Indicators to Columns
    #dataset['Value'] = dataset['Value'].astype(float)
    #dataset = dataset.groupby(['Year', 'Country Name', 'Country Code', 'Indicator Name']).agg({'Value':'sum'}).reset_index().copy()
    #dataset = dataset.pivot(index=['Year', 'Country Name', 'Country Code'], columns='Indicator Name', values='Value').reset_index().copy()

    return dataset

IMF Load Function Test:

In [ ]:
test_IFS = pd.read_csv('IFS sample.csv')
cntrys = ['Afghanistan, Islamic Rep. of', 'Angola']
required_indicators = ['HPHSPA_PA']
test_IFS = clean_IFS_data(test_IFS, countries = cntrys, date_from = 1980, Series_Codes = required_indicators , date_to = 2002, frequency = 'Q' )
test_IFS

In [ ]:
Countries = ['South Africa', 'Thailand', 'Turkey']
IMF_Indicators =['FILR_PA'
                ,'RAXGFX_USD'
                ,'NM_XDC' 
                ,'TMG_CIF_USD'
                ,'TXG_FOB_USD'
                ,'IAP_BP6_USD']
IFS_DB =  pd.read_csv('IFS_timeSeries_1980_2020.csv')
IFS_DB.head()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (5,6,7,8,10,13,14,15,16,18,19,21,22,23,24,25,26,27,28,31,32,35,38,39,40,41,48,49,50,52,54,55,56,57,58,59,60,65,67,69,72,73,74,75,77,78,79,80,81,82,83,86,87,88,89,90,91,92,93,95,96,99,100,101,102,103,106,107,108,109,110,112,113,115,116,120,121,124,125,126,129,130,133,134,137,138,140,141,142,143,146,148,149,150,151,153,154,155,157,158,159,160,161,162,164,165,167,168,171,175,176,177,180,181,184,186,188,191,192,193,194,197,198,200,201,202,204,205,206,207,209,210,211,212,214,215,217,218,219,220,222,223,224,226,227,228,229,230,231,232,233,235,236,237,238,239,240,241,242,243,244,245,248,249,251,252,253,254,255,256,259,260,261,262,263,265,267,269,270,271,272,276,277,278,279,280,282,283,286,287,292,293,294,295,296,297,303,304,305,306,307,309,310,311,312,313,316,320,321,323,324,326,328,329,330,333,334,335,337,338,341,343,344,345,346,347,349,350,351,353,354,355,356,357,358,359,3

,Country Name,Country Code,Indicator Name,Indicator Code,Attribute,1980,1980Q1,1980M1,1980M2,1980M3,...,2020M12,2021,2021Q1,2021M1,2021M2,2021M3,2021Q2,2021M4,2021M5,Base Year
0,"Afghanistan, Islamic Rep. of",512,"Fund Accounts, UFC & Loans, SDRs",HPU_XDR,Value,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Afghanistan, Islamic Rep. of",512,"Fund Accounts, UFC & Loans, SDRs",HPU_XDR,Value,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,281230000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Afghanistan, Islamic Rep. of",512,"Fund Accounts, UFC & Loans, SDRs",HPU_XDR,Value,NaN,NaN,0.0,0.0,0.0,...,281230000.0,NaN,NaN,281230000.0,281230000.0,281230000.0,NaN,278830000.0,NaN,NaN
3,Advanced Economies,110,"Fund Accounts, Outstanding Loans, SDRs",HPL_XDR,Value,30600000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Advanced Economies,110,"Fund Accounts, Outstanding Loans, SDRs",HPL_XDR,Value,NaN,37550000.0,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
IFS_DB.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Attribute,1920,1920Q1,1920Q2,1920Q3,1920Q4,...,2021,2021Q1,2021M1,2021M2,2021M3,2021Q2,2021M4,2021M5,Base Year,Unnamed: 1491
0,"Afghanistan, Islamic Rep. of",512,"Fund Accounts, UFC & Loans, SDRs",HPU_XDR,Value,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Afghanistan, Islamic Rep. of",512,"Fund Accounts, UFC & Loans, SDRs",HPU_XDR,Value,NaN,NaN,NaN,NaN,NaN,...,NaN,281230000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Afghanistan, Islamic Rep. of",512,"Fund Accounts, UFC & Loans, SDRs",HPU_XDR,Value,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,281230000.0,281230000.0,281230000.0,NaN,278830000.0,NaN,NaN,NaN
3,Advanced Economies,110,"Fund Accounts, Outstanding Loans, SDRs",HPL_XDR,Value,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Advanced Economies,110,"Fund Accounts, Outstanding Loans, SDRs",HPL_XDR,Value,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
X = clean_IFS_data(IFS_DB, countries = Countries, date_from = 1980, Series_Codes = IMF_Indicators , date_to = 2002, frequency = 'Q' )
X.head(100)

,Country Name,Country Code,Indicator Name,Year,Value
55,South Africa,199,"International Liquidity, Total Reserves exclud...",1980Q1,NaN
56,South Africa,199,"International Liquidity, Total Reserves exclud...",1980Q1,564000000.0
57,South Africa,199,"International Liquidity, Total Reserves exclud...",1980Q1,NaN
...,...,...,...,...,...
317,Turkey,186,"International Liquidity, Total Reserves exclud...",1980Q2,NaN
318,Turkey,186,"International Liquidity, Total Reserves exclud...",1980Q2,NaN
319,Turkey,186,"Assets, Portfolio Investment, US Dollars - IAP...",1980Q2,NaN


In [ ]:
#X['Value'].str.strip()

#X['Value'] = X['Value'].str.strip().astype(int)



ValueError: cannot convert float NaN to integer

## World Bank API 

In [ ]:
import pandas as pd
import world_bank_data as wb
pd.set_option('display.max_rows', 6)

In [ ]:
wb.get_topics()

In [ ]:
pd.DataFrame(wb.get_series('FM.LBL.BMNY.ZG', 'FM.LBL.BMNY.IR.ZS_DS2', date='2000:2018', id_or_value='id', simplify_index=True)).reset_index()

TypeError: sequence item 4: expected str instance, tuple found

In [ ]:
wb.search_indicators('broad')

## IMF API

In [ ]:
# Import libraries
import requests
import pandas as pd

# URL for the IMF JSON Restful Web Service, 
# IFS database, and Australian export prices series
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/Q.AU.RAXGFX_USD.?startPeriod=1957&endPeriod=2016'

# Get data from the above URL using the requests package
data = requests.get(url).json()

# Load data into a pandas dataframe
auxp = pd.DataFrame(data['CompactData']['DataSet']['Series']['Obs'])

# Show the last five observiations
auxp.tail()

,@TIME_PERIOD,@OBS_VALUE
235,2015-Q4,37492
236,2016-Q1,37700
237,2016-Q2,38976
238,2016-Q3,39649
239,2016-Q4,46416


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5afecb5f-7d28-4b37-ae51-8d94c0fa9663' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>